# Libraries

In [24]:
import os
import glob

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

# Dataset

In [25]:
# list the files in the folder
#!ls ../input/rock-paper-scissors-dataset/Rock-Paper-Scissors

In [26]:
# list the files in the folder
#!ls ../input/rock-paper-scissors-dataset/Rock-Paper-Scissors/test

In [27]:
base_dir = r'C:\Users\himan\Rock Paper Scissors\Rock-Paper-Scissors'

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'test')
test_dir = os.path.join(base_dir, 'validation')

paper_dir = os.path.join(train_dir, 'paper')
rock_dir = os.path.join(train_dir, 'rock')
scissors_dir = os.path.join(train_dir, 'scissors')

paper_imgs = os.listdir(paper_dir)
rock_imgs = os.listdir(rock_dir)
scissors_imgs = os.listdir(scissors_dir)

In [28]:
plt.figure(figsize=(20, 4))
for i, img_path in enumerate(paper_imgs[:5]):
    sp = plt.subplot(1, 5, i+1)
    img = mpimg.imread(os.path.join(paper_dir, img_path))
    plt.imshow(img)
plt.show()

<Figure size 2000x400 with 5 Axes>

In [29]:
plt.figure(figsize=(20, 4))
for i, img_path in enumerate(rock_imgs[:5]):
    sp = plt.subplot(1, 5, i+1)
    img = mpimg.imread(os.path.join(rock_dir, img_path))
    plt.imshow(img)
plt.show()

<Figure size 2000x400 with 5 Axes>

In [30]:
plt.figure(figsize=(20, 4))
for i, img_path in enumerate(scissors_imgs[:5]):
    sp = plt.subplot(1, 5, i+1)
    img = mpimg.imread(os.path.join(scissors_dir, img_path))
    plt.imshow(img)
plt.show()

<Figure size 2000x400 with 5 Axes>

In [31]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2)
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [32]:
BATCH_SIZE = 32
TARGET_SIZE = 64
EPOCHS = 10

In [33]:
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(TARGET_SIZE, TARGET_SIZE), 
                                                    batch_size=BATCH_SIZE, 
                                                    shuffle=True,
                                                    class_mode='categorical')

val_generator = validation_datagen.flow_from_directory(val_dir,
                                                       target_size=(TARGET_SIZE, TARGET_SIZE), 
                                                       batch_size=BATCH_SIZE, 
                                                       shuffle=True,
                                                       class_mode='categorical')

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [34]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

In [35]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 32)         0

In [37]:
model.fit_generator?

In [38]:
history = model.fit_generator(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    verbose=1)

Epoch 1/10
79/79 [==============================] - 15s 188ms/step - loss: 0.6697 - accuracy: 0.7087 - val_loss: 0.4086 - val_accuracy: 0.8199
Epoch 2/10
79/79 [==============================] - 16s 197ms/step - loss: 0.1041 - accuracy: 0.9694 - val_loss: 0.2875 - val_accuracy: 0.8790
Epoch 3/10
79/79 [==============================] - 16s 208ms/step - loss: 0.0561 - accuracy: 0.9849 - val_loss: 0.1965 - val_accuracy: 0.9220
Epoch 4/10
79/79 [==============================] - 17s 210ms/step - loss: 0.0127 - accuracy: 0.9968 - val_loss: 0.2819 - val_accuracy: 0.8978
Epoch 5/10
79/79 [==============================] - 17s 219ms/step - loss: 0.0186 - accuracy: 0.9937 - val_loss: 0.3490 - val_accuracy: 0.8952
Epoch 6/10
79/79 [==============================] - 17s 215ms/step - loss: 0.0066 - accuracy: 0.9980 - val_loss: 0.4111 - val_accuracy: 0.8737
Epoch 7/10
79/79 [==============================] - 18s 225ms/step - loss: 0.0023 - accuracy: 0.9996 - val_loss: 0.4098 - val_accuracy: 0.8763

In [39]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.savefig('./foo.png')
plt.show()

<Figure size 1400x500 with 2 Axes>

In [40]:
from tensorflow.keras.models import load_model

model.save('rock_paper_scissors_cmp.h5')

In [46]:
import gradio 
import requests
import urllib
import tensorflow as tf
from PIL import Image




model=tf.keras.models.load_model(r'C:\Users\himan\Rock Paper Scissors\rock_paper_scissors_cmp.h5')
labels=['Paper','Rock','Scissors']

def classify_image(inp):
    # Resize the input image to (64, 64)
    inp = Image.fromarray(inp)
    inp = inp.resize((64, 64))
    inp = np.array(inp)

    # Preprocess the input image
    inp = inp[None, ...]
    inp = tf.keras.applications.inception_v3.preprocess_input(inp)

    # Make a prediction using the model
    prediction = model.predict(inp).flatten()
    return {labels[i]: float(prediction[i]) for i in range(no_classes)}


import gradio
import numpy as np
from PIL import Image
import tensorflow as tf

def classify_image(inp):
    if inp is None:
        # Handle the case where inp is None
        return {}
    # Resize the input image to (64, 64)
    inp = Image.fromarray(inp)
    inp = inp.resize((64, 64))
    inp = np.array(inp)

    # Preprocess the input image
    inp = inp[None, ...]
    inp = tf.keras.applications.inception_v3.preprocess_input(inp)

    # Make a prediction using the model
    prediction = model.predict(inp).flatten()
    no_classes = 3  # Set this to the number of classes in your model's output
    return {labels[i]: float(prediction[i]) for i in range(no_classes)}





In [47]:

image = gradio.components.Image(shape=(224, 224))
label = gradio.components.Label(num_top_classes=3)

gradio.Interface(fn=classify_image, inputs=image, outputs=label, interpretation="default").launch(share=True)


Running on local URL:  http://127.0.0.1:7866

Could not create share link. Missing file: C:\Users\himan\anaconda3\lib\site-packages\gradio\frpc_windows_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.2
3. Move the file to this location: C:\Users\himan\anaconda3\lib\site-packages\gradio


1/1 [==============================] - 0s 39ms/step
